# Fuzzy Matching with RL - 247 & Rivals

> Leveraging the RL library to determine approximate matching over a range of fields using various string methods methods.  This specifically focuses on 247 & Rivals first.

In [1]:
import json
import pandas
import time
import os
import recordlinkage
import csv
import core_constants as cc
import functions as fx
import sqlite3 as sql

#not currently using jellyfish
import jellyfish as jf

## Load and Merge Source Files Then Create a List of Dicts for each Dataset
> This was originally set up for all of the keys in the sourcefiles.json config.  Since this file is currently only going to serve 247 & Rivals, I've hardcoded the keys to fetch

In [ ]:
dataset_keys = {'sports247': 1, 'rivals': 2}
for key,value in dataset_keys.items():
    SQL = """SELECT IDYR, College, Year, PlayerName, HighSchool, City, State, Position FROM SourcedPlayers WHERE KeyDataset = {}""".format(value)
    vars()[key] = (fx.connDBAndReturnDF(SQL)).set_index('IDYR')
    (vars()[key]).index.name = key + '_IDYR'
    (vars()[key])['IDYR'] = (vars()[key]).index.get_level_values(0)

In [2]:
dfFinal = fx.doFuzzyMatching('Sports247', 'NFL')
dfFinal.head()

,ID,PlayerName,Position,sum,sourceID,targetID
"(aaironsavage_auburn_2005, carltondavis_auburn)",0.578947,0.333333,1,0.637427,aaironsavage_auburn_2005,carltondavis_auburn
"(aaronbaltazar_colorado_2015, chrisbrown_colorado)",0.500000,0.153846,1,0.551282,aaronbaltazar_colorado_2015,chrisbrown_colorado
"(aaronbedgood_coastalcarolina_2019, jeromesimpson_coastalcarolina)",0.655172,0.230769,1,0.628647,aaronbedgood_coastalcarolina_2019,jeromesimpson_coastalcarolina
"(aaronbenson_texas_2010, malikjefferson_texas)",0.550000,0.357143,1,0.635714,aaronbenson_texas_2010,malikjefferson_texas
"(aaronbentley_memphis_2003, clintonmcdonald_memphis)",0.478261,0.200000,1,0.559420,aaronbentley_memphis_2003,clintonmcdonald_memphis


## Create Blockers
> I swear this isn't working.  And honestly I guess I'm ok with it not working since I'm doing string operations later on this script.  This used to take longer than 20 minutes but has been fixed.

In [ ]:
indexer = recordlinkage.BlockIndex(on=['College', 'Year'])
candidate_links = indexer.index(sports247, rivals)

## Define Features & Generate Comparison Vector Set

> These are all pretty straightforward.  The toughest one to assess is position - since the services don't always categorize players in the same way or have the same abbreviation for a single position.  Since this is often only a 2 or 3 letter string, I decided to do an exact match.

In [ ]:
c = recordlinkage.Compare()

c.exact('IDYR', 'IDYR', label='IDYR')
c.string('PlayerName', 'PlayerName', method='damerau_levenshtein', label='PlayerName')
c.string('City', 'City', label='City')
c.exact('State', 'State', label='State')
c.string('HighSchool', 'HighSchool', label='HighSchool')
c.exact('Position', 'Position', label='Position')

try:
    features = c.compute(candidate_links, sports247, rivals)
except KeyError as e:
    print(e)

## Create Sum
> Final value will be between 0 and 1.  Sum represents a %confidence% level during fuzzy matching. Since an exact match on ID dictates a 100% confident match, I've removed that column from the sum.

In [ ]:
features['Sum'] = (features['PlayerName'] + features['City'] + features['State'] + features['HighSchool']  + features['Position'])/5

In [ ]:
features.head()

## Filter by Threshold

> Agreed upon confidence level for 247 & Rivals is .6943.  This takes longer than expected.  

In [ ]:
count = 0
filteredList = []
noMatch = []

features['sports247_IDYR'] = features.index.get_level_values(0)
features['rivals_IDYR'] = features.index.get_level_values(1)

for idx, data in features.groupby(level=0):
    data = data.loc[data['Sum'].idxmax()]
    if (data['IDYR'] == 1):
        filteredList.append(data)
    elif (data['IDYR'] != 1 and data['Sum'] > .6943):
        filteredList.append(data)
    else:
        noMatch.append(data)

## Convert Series to Dataframe
> output of the above is a Pandas Series and needs to be pushed to a dataframe to be accessible

In [1]:
dfFinal = pandas.DataFrame()
dfFinal = dfFinal.append(filteredList)
dfFinal.sort_values(by='SUM')

NameError: name 'pandas' is not defined

In [ ]:
dict_dfFinal = dfFinal.to_dict('records')

In [ ]:
dict_dfFinal[0]

In [ ]:
for record in dict_dfFinal:
    Values = [record['sports247_IDYR'], record['rivals_IDYR'], 2, 3, record['Sum']]
    query = '''INSERT INTO RecordLinks(MasterID, TargetID, KeyDataSet, KeyLinkType, LinkConfidence)
        VALUES (?,?,?,?,?)'''
    
    conn = sql.connect(cc.databaseName)
    c = conn.cursor()
    
    c.execute(query, Values)
    conn.commit()
    
conn.close()